In [ ]:
import re
from pathlib import Path

from pydantic import BaseModel

In [ ]:
class Machine(BaseModel):
    """Represents a machine with a target value, buttons, and joltage ratings."""

    target: int
    buttons: list[int]
    joltages: list[int]

In [ ]:
lines = Path("day10_input.txt").read_text().splitlines()

machines: list[Machine] = []
for line in lines:
    # Target:
    if match := re.match(r"\[(.+)\]", line):
        target = int(
            "".join(reversed(match.group(1).replace(".", "0").replace("#", "1"))), 2
        )
    # Buttons:
    matches = re.findall(r"\(([^)]*)\)", line)
    buttons = [sum(1 << b for b in map(int, x.split(","))) for x in matches]
    # Joltage:
    if match := re.search(r"\{(.+)\}", line):
        joltages = [int(x) for x in match.group(1).split(",")]
    machines.append(Machine(target=target, buttons=buttons, joltages=joltages))

# Part 1

We can see this as a graph where we start at state 0 and want to reach the target state.
We can use BFS to explore all reachable states. From each state, there are N neighbors,
one for each button. Each button press is a transition that toggles certain bits in the
current state.


In [ ]:
button_presses = 0

for i, machine in enumerate(machines):
    queue = [(0, 0)]  # (state, steps)
    visited = set()
    while queue:
        state, steps = queue.pop(0)
        if state == machine.target:
            button_presses += steps
            break
        for button in machine.buttons:
            new_state = state ^ button
            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, steps + 1))
    else:
        print(f"Machine {i}: Target {machine.target:08b} not reachable")

button_presses

# Part 2

The search space is too large to explore exhaustively with BFS. We can instead view this
as an equation $Ax = t$, where $A$ is a matrix representing the button toggles, $x$ is a
vector of button presses, and $t$ is the target state. We want to solve this for $x$.

The MILP solver in SciPy handles problems of this form:

$$
\begin{aligned}
\min_x \quad & c^{\mathsf T} x \\
\text{subject to} \quad & b_\ell \le Ax \le b_u, \\
& \ell \le x \le u, \\
& x_i \in \mathbb{Z}, \quad i \in X_i .
\end{aligned}
$$

In our case:

- $c$ is a vector of ones (we minimize the sum of button presses)
- $A$ is the button toggle matrix
- $b_\ell$ and $b_u$ are both the target state vector, since we want $Ax = t$
- $\ell = 0$ and $u = \infty$, which are also the default values
- $X_i$ is the set of all indices (all $x_i$ are integers)


In [ ]:
import numpy as np
from scipy.optimize import LinearConstraint, milp


def solve_machine(machine: Machine) -> int:
    """Solve Ax = t for minimum sum of x_j using MILP."""
    target = machine.joltages
    buttons = machine.buttons

    # Dimensions
    d = len(target)
    m = len(buttons)

    # Build A matrix (d x m)
    # Each button is a binary number; bit i set means it toggles counter i
    A = np.zeros((d, m))  # noqa: N806
    for j, button in enumerate(buttons):
        for i in range(d):
            if button & (1 << i):  # bit i is set
                A[i, j] = 1

    # Objective: minimize sum of elements in x
    c = np.ones(m)

    # Constraints on x: A @ x = t
    constraint = LinearConstraint(A, target, target)

    # All elements of x must be integers
    integrality = np.ones(m)

    # Run the solver
    result = milp(c=c, constraints=constraint, integrality=integrality)
    if not result.success:
        raise RuntimeError("MILP failed")

    return int(result.x.sum())

In [ ]:
sum(solve_machine(m) for m in machines)